In [2]:
import pandas as pd
from openpyxl import Workbook
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import *
from openpyxl.chart import *
from openpyxl.chart.shapes import GraphicalProperties
from openpyxl.chart.label import DataLabelList

import string
import os

In [3]:
input_file = 'input_data/supermarket_sales.xlsx'
output_file = 'output_data/report_2019_jupiter.xlsx'

In [23]:
df = pd.read_excel(input_file)
df[['Gender', 'Product line', 'Total']].tail()

,Gender,Product line,Total
995,Male,Health and beauty,42.3675
996,Female,Home and lifestyle,1022.4900
997,Male,Food and beverages,33.4320
998,Male,Home and lifestyle,69.1110
999,Female,Fashion accessories,649.2990


In [24]:
df1 = df.pivot_table(index='Gender',
                    columns='Product line',
                    values='Total',
                    aggfunc='sum').round(0)

In [6]:
df1

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Gender,,,,,,
Female,27102.0,30437.0,33171.0,18561.0,30037.0,28575.0
Male,27236.0,23868.0,22974.0,30633.0,23825.0,26548.0


In [25]:
df1.to_excel(output_file,
            sheet_name='Product line',
            startrow=4)

In [26]:
wb = load_workbook(output_file)
wb.active = wb['Product line']

min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

In [27]:
print(min_column,max_column,min_row,max_row)

1 7 5 7


In [28]:
wb = load_workbook(output_file)
wb.active = wb['Product line']
sheet = wb['Product line']

# barchart
barchart = BarChart()

#locate data and categories
data = Reference(sheet,
                 min_col=min_column+1,
                 max_col=max_column,
                 min_row=min_row,
                 max_row=max_row) 
categories = Reference(sheet,
                       min_col=min_column,
                       max_col=min_column,
                       min_row=min_row+1,
                       max_row=max_row) 

# adding data and categories
barchart.add_data(data, titles_from_data=True)
barchart.set_categories(categories)

#location chart
sheet.add_chart(barchart, "B12")
barchart.title = 'Sales by Product line'
barchart.style = 2
wb.save(output_file)

In [29]:
import string
alphabet = list(string.ascii_uppercase)
excel_alphabet = alphabet[0:max_column] 
print(excel_alphabet)

['A', 'B', 'C', 'D', 'E', 'F', 'G']


In [30]:
wb = load_workbook(output_file)
sheet = wb['Product line']
# sum in columns B-G
for i in excel_alphabet:
    if i!='A':
        sheet[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})'
        sheet[f'{i}{max_row+1}'].style = 'Currency'
# adding total label
sheet[f'{excel_alphabet[0]}{max_row+1}'] = 'Total'
wb.save(output_file)

In [51]:
wb = load_workbook(output_file)
sheet = wb['Product line']
sheet['A1'] = 'Sales Report'
sheet['A2'] = '2019'
sheet['A1'].font = Font('Arial', bold=True, size=20)
sheet['A2'].font = Font('Arial', bold=True, size=12)
wb.save(output_file)

Workbook 2

In [40]:
df2 = df[['City', 'Total']].groupby(['City']).sum().round()

In [42]:
df2['percent'] = ((df2['Total'] / df2['Total'].sum()) * 100).round(0)

In [43]:
del df2['Total']
df2

,percent
City,
Mandalay,33.0
Naypyitaw,34.0
Yangon,33.0


In [44]:
df2.to_excel(output_file,
            sheet_name= 'City',
            startrow =4)

In [45]:
with pd.ExcelWriter(output_file) as writer:  
    df1.to_excel(writer, sheet_name='Product line', startrow=4)
    df2.to_excel(writer, sheet_name='City', startrow=4)

In [46]:
wb = load_workbook(output_file)
wb.active = wb['City']

min_column_2 = wb.active.min_column
max_column_2 = wb.active.max_column
min_row_2 = wb.active.min_row
max_row_2 = wb.active.max_row

In [47]:
print(min_column_2,max_column_2,min_row_2,max_row_2)

1 2 5 8


In [49]:
wb = load_workbook(output_file)
sheet_2 = wb['City']

piechart = PieChart3D()

data_2 = Reference(sheet_2,
                min_col=min_column_2+1,
                max_col=max_column_2,
                min_row=min_row_2,
                max_row=max_row_2)
categories_2 = Reference(sheet_2,
                       min_col=min_column_2,
                       max_col=min_column_2,
                       min_row=min_row_2+1,
                       max_row=max_row_2)

piechart.add_data(data_2, titles_from_data=True)
piechart.set_categories(categories_2)
piechart.dataLabels = DataLabelList() 
piechart.dataLabels.showVal = True  

sheet_2.add_chart(piechart, "B12")
piechart.title = 'Sales by Region'
piechart.style = 2 #
wb.save(output_file)

In [55]:
def automate_excel(file_name, sheet_name, index, chart_style, columns=None):
    exel_file = pd.read_excel(input_file)
    df1 = exel_file.pivot_table(index='Gender', columns='Product line', values='Total', aggfunc='sum').round()

    try:
        with pd.ExcelWriter(output_file, mode='a', if_sheet_exists='replace') as writer:
            df1.to_excel(writer, sheet_name='Product line', startrow=4)
    except KeyError:
        with pd.ExcelWriter(output_file) as writer:
            df1.to_excel(writer, name_sheet='Product line', startrow=4)

    wb = load_workbook(output_file)
    if 'Sheet' in wb.sheetnames:
        wb.remove(wb['Sheet'])
    wb.active = wb['Product line']
    sheet = wb['Product line']

    min_column = wb.active.min_column
    max_column = wb.active.max_column
    min_row = wb.active.min_row
    max_row = wb.active.max_row

    barchart = BarChart()
    
    data = Reference(sheet,
                    min_col=min_column+1,
                    max_col=max_column,
                    min_row=min_row,
                    max_row=max_row) 
    categories = Reference(sheet,
                        min_col=min_column,
                        max_col=min_column,
                        min_row=min_row+1,
                        max_row=max_row) 
   
    barchart.add_data(data, titles_from_data=True)
    barchart.set_categories(categories)

    sheet.add_chart(barchart, "B12")
    barchart.title = 'Sales by Product line'
    barchart.style = 2
    wb.save(output_file)

    alphabet = list(string.ascii_uppercase)
    excel_alphabet = alphabet[0:max_column] 

    for i in excel_alphabet:
        if i!='A':
            sheet[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})'
            sheet[f'{i}{max_row+1}'].style = 'Currency'
    sheet[f'{excel_alphabet[0]}{max_row+1}'] = 'Total'

    sheet['A1'] = 'Sales Report'
    sheet['A2'] = '2019'
    sheet['A1'].font = Font('Arial', bold=True, size=20)
    sheet['A2'].font = Font('Arial', bold=True, size=12)
    wb.save(output_file)

In [56]:
automate_excel(input_file, 'Product line', 'Gender', BarChart(), 'Product line')

In [57]:
automate_excel(input_file, 'City', 'City', PieChart3D())